In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [2]:
from datasets import load_dataset

dataset = load_dataset('Prajapat/medical_datset',split = 'test')


In [3]:
dataset

Dataset({
    features: ['Prompt', 'Symptoms'],
    num_rows: 595
})

In [4]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:3", 
    trust_remote_code=False,
    revision="main")

CUDA extension not installed.
CUDA extension not installed.
/DATA/sriparna/TA/Daivik/.conda/lib/python3.9/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'mode

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True,device_map="cuda:3")


In [6]:
intstructions_string = f"""You are a medical expert.Your task is to find solution  for a given Problem.

Please find Symptoms to the following Prompt.
"""

prompt_template = lambda Symptoms: f'''[INST] {intstructions_string} \n{prompt} \n[/INST]'''

In [7]:
for i in dataset['Prompt'][:5]:
  prompt = i
  print(prompt_template(prompt))
  
  inputs = tokenizer(prompt_template(i), return_tensors="pt")
  outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                            max_new_tokens=200)

  result = tokenizer.batch_decode(outputs)[0]
  print(result)

[INST] You are a medical expert.Your task is to find solution  for a given Problem.

Please find Symptoms to the following Prompt.
 
What are the symptoms of Dysfunctional uterine bleeding? 
[/INST]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:3 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

: 